<a href="https://colab.research.google.com/github/HILALOZTEMEL/PyTorch/blob/main/13_Training_Convolutional_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


**Dataloaders**

In [12]:
transform = transforms.Compose(    
    [transforms.ToTensor(),     
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,                                        
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                        shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


def __init__(self,num_classes=10):

Evrişimsel filtreler için kanal sayısı isteğe beğlıdır.Tecrübe ile doğru seçimleri yapma konusunda giderek daha iyi olacaksınız.

Evrişimsel filtrelerde ilk filtrenin 32 ,ikinci filtrenin 64 ve üçüncü filtrenin 128 kanalolması gerektiğine karar veriyoruz.Kanal sayısını kademeli olarak arttırmak çok yaygındır.

Ayrıca her evrişim katmanından sonra havuz katmanlarına sahip olmaya karar veriyoruz.

Sonunda tam bağlantılı bir katman ilave ediyoruz.İlk boyutu , son katmanın sahip olduğu birim sayısıdır.

Bu durumda 128(derinlik veya kanal sayısı için) çarpı 4(yükseklik için ) çarpı genişlik için 4 'tür.

4,conv filtrelerinden sonra uyguladığımız havuzların her biri için 32'yi 2'ye üç kez bölmekten gelir.

In [13]:
class Net(nn.Module):
  def __init__(self, num_classes=10):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
            self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)        
            self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)        
            self.pool = nn.MaxPool2d(2, 2)        
            self.fc = nn.Linear(128 * 4 * 4, num_classes)
  def forward(self, x):        
            x = self.pool(F.relu(self.conv1(x)))        
            x = self.pool(F.relu(self.conv2(x)))        
            x = self.pool(F.relu(self.conv3(x)))        
            x = x.view(-1, 128 * 4 * 4)
            return self.fc(x)



def forward(self,x):

Girişe önce conv filtresini uyguluyoruz ,ardından relu ve havuzlama katmanı tarafından (relu işlevsel yolu nasıl kullandığımıza dikkat edin.)

Daha sonra ağın ikinci ve üçüncü katmanı için aynı işlemi yapıyoruz.

Son olarak, ağın üç boyutunu da sıkıştırarak ağı tam bağlı katman için hazırlıyoruz.derinlik=128,genişlik=4,yükseklik=4 bir boyutta ve ardından tam bağlantılı katmanı uygularız.



In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


**Optimizer and Loss Function**

Daha sonra nesneyi classnet'ten başlatırız ve somutlaştırırız.

Çapraz entropi kaybı işlevi ve öğrenme oranı 3e-4 olan Adam Optimizer.

Ağı eğitmek için, tren yükleyicideki tüm veriler üzerinde birden çok kez , bu durumda 10 kez döngü yaparız.

In [14]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=3e-4)


**Training a CNN**

In [15]:
for epoch in range(10):  
  for i, data in enumerate(trainloader, 0):
    # Get the inputs        
    inputs, labels = data
    
    # Zero the parameter gradients        
    optimizer.zero_grad()
    
    # Forward + backward + optimize        
    outputs = net(inputs)        
    loss = criterion(outputs, labels)        
    loss.backward()        
    optimizer.step()
    print('Finished Training')
  

Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished Training
Finished T

tahminlere ve etiketlere dayanarak kayıp fonksiyonu hesaplarız.sonra degradeleri geri kullanarak hesaplarız ve son olarak optimize edicimizi kullanara ağırlıkları güncelleriz.

**Evaluating the results**

sonuçları değerlendirmek, tamamenbağlı ağları kullanmak kadar basittir.net.eval() moduna koyarız ve her görüntü için ağı tahminlerde bulunmak için kullanırız.

'100*correct / total' doğru tahminlerin sayısını toplam puan sayısına böleriz. test seti ve sonuçları yazdırıyoruz , bu durumda %68 doğruluk elde ediyoruz.

In [16]:
correct,total= 0, 0
predictions=[]
net.eval()
for i, data in enumerate(testloader,0):
  inputs,labels = data
  outputs = net(inputs)
  _, predicted = torch.max(outputs.data ,1 )
  predictions.append(outputs)
  total += labels.size(0)
  correct += (predicted==labels).sum().item()

print('The testing set accuracy of the network is: %d %%' % (100*correct / total))  

The testing set accuracy of the network is: 71 %
